<a href="https://colab.research.google.com/github/Sillians/Basil-guide/blob/master/Handling_Missing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DATASET FROM HOUSE PRICING COMPETITION FOR KAGGLE LEARNERS

In [0]:
# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex2 import *
print("Setup Complete")

In [0]:
# Import Libraries

import pandas as pd
from sklearn.model_selection import train_test_split

X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,1774,1822,1828,0,0,1828,0,0,2,0,3,1,9,1,2007.0,3,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,894,894,894,0,0,894,0,0,1,0,2,1,5,0,1962.0,1,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,163,876,964,0,0,964,1,0,1,0,2,1,5,0,1921.0,2,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,350,1568,1689,0,0,1689,1,0,2,0,3,1,7,2,2002.0,3,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,1541,1541,1541,0,0,1541,0,0,2,0,3,1,6,1,2001.0,3,843,468,81,0,0,0,0,0,1,2006


In [9]:
y_train.head()

Id
619    314813
871    109500
93     163500
818    271000
303    205000
Name: SalePrice, dtype: int64

You can already see a few missing values in the first several rows.  In the next step, you'll obtain a more comprehensive understanding of the missing values in the dataset.

**Step 1: Preliminary investigation**

Run the code cell below without changes.

In [10]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


    To compare different approaches to dealing with missing values, you'll use the same `score_dataset()` function. 
    
    This function reports the [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) from a random forest model.

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


# Function for comparing different approaches
def score_dataset(X_train, X_val, y_train, y_val):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return mean_absolute_error(y_val, preds)

### **Step 2: Drop columns with missing values**

    In this step, you'll preprocess the data in `X_train` and `X_val` to remove columns with missing values.  Set the preprocessed DataFrames to `reduced_X_train` and `reduced_X_val`, respectively.

In [0]:
# Get names of columns with missing values
cols_missing = [cols for cols in X_train.columns
               if X_train[cols].isnull().any()]

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(cols_missing, axis=1)
reduced_X_val = X_val.drop(cols_missing, axis=1)


In [14]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_val, y_train, y_val))

MAE (Drop columns with missing values):
17837.82570776256


### Step 3: Imputation

### Part A

Use the next code cell to impute missing values with the mean value along each column.  Set the preprocessed DataFrames to `imputed_X_train` and `imputed_X_val`.  Make sure that the column names match those in `X_train` and `X_val`.

In [0]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
db_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(db_imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(db_imputer.transform(X_val))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns


In [17]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_val, y_train, y_val))

MAE (Imputation):
18062.894611872147


### Step 4: Generate test predictions

In this final step, you'll use any approach of your choosing to deal with missing values.  Once you've preprocessed the training and validation features, you'll train and evaluate a random forest model.  Then, you'll preprocess the test data before generating predictions that can be submitted.

In [0]:
# Preprocessed training and validation features
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_val = pd.DataFrame(final_imputer.transform(X_val))

# Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_val.columns = X_val.columns

In [20]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_val = model.predict(final_X_val)
print("MAE (Your appraoch):")
print(mean_absolute_error(y_val, preds_val))

MAE (Your appraoch):
17791.59899543379


**Part B**

    Use the next code cell to preprocess your test data.  Make sure that you use a method that agrees with how you preprocessed the training and validation data, and set the preprocessed test features to `final_X_test`.

    Then, use the preprocessed test features and the trained model to generate test predictions in `preds_test`.



In [0]:
# Preprocess text data
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)

In [0]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

In [23]:
output

,Id,SalePrice
0,1461,125985.50
1,1462,154599.50
2,1463,180070.24
3,1464,183544.50
4,1465,197549.92
5,1466,185360.97
6,1467,169198.67
7,1468,174488.60
8,1469,178512.30
9,1470,123798.08
